In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.linalg import inv
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from shapely import geometry as geom
import geopandas as gpd

In [2]:
def create_queen_adjacency(n):
    """
    Create a queen adjacency matrix for an n x n grid.
    Each cell is connected to its 8 adjacent neighbors (N, S, E, W, NE, NW, SE, SW).
    
    Parameters:
    -----------
    n : int
        Size of the grid (n x n)
        
    Returns:
    --------
    W : scipy.sparse.csr_matrix
        Sparse adjacency matrix (n² x n²)
    """
    W = sp.lil_matrix((n * n, n * n))
    for row in range(n):
        for col in range(n):
            idx = row * n + col
            
            # Cardinal directions (same as rook)
            if row > 0:  # North
                W[idx, (row - 1) * n + col] = 1
            if row < n - 1:  # South
                W[idx, (row + 1) * n + col] = 1
            if col > 0:  # West
                W[idx, row * n + (col - 1)] = 1
            if col < n - 1:  # East
                W[idx, row * n + (col + 1)] = 1
                
            # Diagonal connections (unique to queen)
            if row > 0 and col > 0:  # Northwest
                W[idx, (row - 1) * n + (col - 1)] = 1
            if row > 0 and col < n - 1:  # Northeast
                W[idx, (row - 1) * n + (col + 1)] = 1
            if row < n - 1 and col > 0:  # Southwest
                W[idx, (row + 1) * n + (col - 1)] = 1
            if row < n - 1 and col < n - 1:  # Southeast
                W[idx, (row + 1) * n + (col + 1)] = 1
    
    # Return as CSR format for efficient operations
    return W.tocsr()

In [3]:
w = create_queen_adjacency(3)

In [6]:
print(w.toarray())

[[0. 1. 0. 1. 1. 0. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 0. 0. 0.]
 [0. 1. 0. 0. 1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 1. 0. 1. 1. 0.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 0. 1. 0. 0. 1. 1.]
 [0. 0. 0. 1. 1. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1. 1. 1. 0. 1.]
 [0. 0. 0. 0. 1. 1. 0. 1. 0.]]


In [16]:
def row_standardize_sparse(W):
    """
    Row-standardize a sparse matrix efficiently.
    
    Parameters:
    -----------
    W : scipy.sparse.csr_matrix
        Input sparse matrix
        
    Returns:
    --------
    W_std : scipy.sparse.csr_matrix
        Row-standardized sparse matrix
    """
    W_csr = W.tocsr()
    row_sums = np.array(W_csr.sum(axis=1)).flatten()
    # Handle zero row sums to avoid division by zero
    # row_sums[row_sums == 0] = 1
    
    # Create diagonal matrix with inverse of row sums
    D_inv = sp.diags(1.0 / row_sums)
    
    # Multiply to get row-standardized matrix
    W_std = D_inv @ W_csr
    return W_std

In [17]:
w_std = row_standardize_sparse(w)

In [18]:
print(w_std.toarray())

[[0.         0.33333333 0.         0.33333333 0.33333333 0.
  0.         0.         0.        ]
 [0.2        0.         0.2        0.2        0.2        0.2
  0.         0.         0.        ]
 [0.         0.33333333 0.         0.         0.33333333 0.33333333
  0.         0.         0.        ]
 [0.2        0.2        0.         0.         0.2        0.
  0.2        0.2        0.        ]
 [0.125      0.125      0.125      0.125      0.         0.125
  0.125      0.125      0.125     ]
 [0.         0.2        0.2        0.         0.2        0.
  0.         0.2        0.2       ]
 [0.         0.         0.         0.33333333 0.33333333 0.
  0.         0.33333333 0.        ]
 [0.         0.         0.         0.2        0.2        0.2
  0.2        0.         0.2       ]
 [0.         0.         0.         0.         0.33333333 0.33333333
  0.         0.33333333 0.        ]]


In [19]:
np.random.seed(42)
n = w.shape[0]

In [20]:
n

9

In [21]:
I = sp.identity(n, format='csr')

In [22]:
print(I.toarray())

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [23]:
epsilon = np.random.normal(0, 1, size=n)

In [24]:
epsilon

array([ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986, -0.23415337,
       -0.23413696,  1.57921282,  0.76743473, -0.46947439])